In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path

2023-10-26 10:10:29.719041: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 10:10:30.300823: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['gender']
    return ecg_data, label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-26 10:10:34.625513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31042 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:18:00.0, compute capability: 7.0


In [3]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [4]:
model = keras.models.load_model("data/models/resnet-age.keras")

In [5]:
for i in range(len(model.layers)):
    model.layers[i].trainable = False

In [6]:
x = model.layers[-2].output
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name = "tl_dense_3")(x)

new_model = keras.Model(inputs=model.input,outputs=x)

new_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [7]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "cnn-tl-gender"))
]

new_model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-26 10:10:43.329420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-26 10:10:45.131210: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f12dd8439d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-26 10:10:45.131244: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-10-26 10:10:45.199973: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-26 10:10:45.654748: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   7851/Unknown - 231s 29ms/step - loss: 0.5096 - accuracy: 0.7516 - auc: 0.8298INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


7851/7851 [==============================] - 282s 35ms/step - loss: 0.5096 - accuracy: 0.7516 - auc: 0.8298 - val_loss: 0.5158 - val_accuracy: 0.7449 - val_auc: 0.8250 - lr: 0.0010
Epoch 2/10
7849/7851 [============================>.] - ETA: 0s - loss: 0.4934 - accuracy: 0.7608 - auc: 0.8409INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


...

7851/7851 [==============================] - 276s 35ms/step - loss: 0.4461 - accuracy: 0.7921 - auc: 0.8735 - val_loss: 0.5785 - val_accuracy: 0.7268 - val_auc: 0.8086 - lr: 0.0010
Epoch 9/10
7849/7851 [============================>.] - ETA: 0s - loss: 0.4432 - accuracy: 0.7926 - auc: 0.8753INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


7851/7851 [==============================] - 276s 35ms/step - loss: 0.4431 - accuracy: 0.7927 - auc: 0.8753 - val_loss: 0.6239 - val_accuracy: 0.6992 - val_auc: 0.7941 - lr: 0.0010
Epoch 10/10
7849/7851 [============================>.] - ETA: 0s - loss: 0.4395 - accuracy: 0.7957 - auc: 0.8776INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


INFO:tensorflow:Assets written to: data/models/cnn-tl-gender-2023-10-26_10-10-41/assets


7851/7851 [==============================] - 276s 35ms/step - loss: 0.4395 - accuracy: 0.7957 - auc: 0.8776 - val_loss: 0.6084 - val_accuracy: 0.7102 - val_auc: 0.7937 - lr: 0.0010
